In [47]:
import pickle
import numpy as np
import plotly.graph_objects as go
from classes_colors import classes, colors

In [102]:
with open("sensor_labels.pkl", "rb") as f:
    sensor_labels = pickle.load(f)

with open("sensor_data_imputed.pkl", "rb") as f:
    sensor_data_imputed = pickle.load(f)

In [107]:
def get_sensor_data(sensor_data, day, matrix, sensor):
    data_lengths = []
    data_arr = []
    for heater_idx in range(10):
        dates = sensor_data[f"day{day}"][f"mat{matrix}"][sensor][heater_idx]["Date"].to_numpy()
        resistance = sensor_data[f"day{day}"][f"mat{matrix}"][sensor][heater_idx]["Resistance Gassensor"].to_numpy()
        data_lengths.append(len(dates))
        data_arr.append({"dates": dates, "resistance": resistance})

    min_length = min(data_lengths)
    for i in range(len(data_arr)):
        data_arr[i]["dates"] = data_arr[i]["dates"][:min_length]
        data_arr[i]["resistance"] = data_arr[i]["resistance"][:min_length]

    return np.array(data_arr)

In [104]:
def plot_data_arr(data_arr, title, labels):
    fig = go.Figure()

    for i, data in enumerate(data_arr):
        fig.add_trace(go.Scatter(x=data["dates"],
                                 y=data["resistance"],
                                 mode="lines",
                                 name=f"Step {i}"))
    
    for label in labels:
        fig.add_vrect(x0=label["start"],
                      x1=label["end"],
                      annotation_text=classes[label["label"]],
                      annotation_position="top left",
                      fillcolor=colors[label["label"]],
                      opacity=0.2,
                      line_width=0)

    fig.update_layout(title=title, title_x=0.5, width=1000, height=600)

    fig.update_yaxes(type="log")
    fig.show()

In [105]:
day = 1
matrix = 1
for sensor in range(2):
    title = f"Day {day}, Matrix {matrix}, Sensor {sensor}"
    data_arr = get_sensor_data(sensor_data_imputed, day, matrix, sensor)
    plot_data_arr(data_arr, title, sensor_labels[f"day{day}"][f"mat{matrix}"])

In [114]:
def build_sensor_cls_data(data_arr, sensor_labels, day, matrix):
    X = [[],[]]
    y = []
    for heater_data in data_arr:
        labels = sensor_labels[f"day{day}"][f"mat{matrix}"]
        for label in labels:
            start = np.datetime64(label["start"])
            end = np.datetime64(label["end"])
            cls = label["label"]
            cond = np.logical_and(heater_data["dates"] >= start, heater_data["dates"] <= end)
            dates = heater_data["dates"][cond]
            resistances = heater_data["resistance"][cond]
            X[0].extend(dates)
            X[1].extend(resistances)
            y.extend([cls] * len(dates))
            print(len(dates))
        print("-" * 10)
    X = np.array(X)
    y = np.array(y)
    return X, y

day = 1
matrix = 1
sensor = 0
data_arr = get_sensor_data(sensor_data_imputed, day, matrix, sensor)
X, y = build_sensor_cls_data(data_arr, sensor_labels, day, 1)

77
78
100
94
----------
77
78
100
94
----------
77
78
100
93
----------
78
77
99
94
----------
78
77
99
94
----------
78
77
99
94
----------
78
77
99
94
----------
77
77
99
94
----------
77
78
99
94
----------
77
78
99
94
----------


In [116]:
print(X.shape, y.shape)

(2, 3481) (3481,)


In [118]:
X[1]

array([6412524.5, 6408762.0, 6338104.5, ..., 2916340.25, 2876404.5,
       2903934.75], dtype=object)